## Individual run

In [2]:
import pickle

In [3]:
import sys
sys.path.append('../')
from util import corpus, tokenize, util, alignment, word2vec, retrieval

In [4]:
df = util.decompress_pickle('../../data/word2vec/epo_en/corpus/epo_10_20_df_wo_pre')
df

,name,titles,en,de
0,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,A modular power converter comprising a thermal...,"Modularer Leistungswandler, der Folgendes umfa..."
1,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,"The modular power converter of claim 1, wherei...","Modularer Leistungswandler nach Anspruch 1, wo..."
2,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,"The modular power converter of claim 1, wherei...","Modularer Leistungswandler nach Anspruch 1, wo..."
3,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,"The modular power converter of claim 1, wherei...","Modularer Leistungswandler nach Anspruch 1, wo..."
4,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,"The modular power converter of claim 1, wherei...","Modularer Leistungswandler nach Anspruch 1, wo..."
...,...,...,...,...
545093,EP17000010NWB1.xml,de\tSTREULICHTBLENDE EINES BILDERFASSUNGSGERÄT...,"Lens hood according to Claim 10, characterized...","Streulichtblende nach Anspruch 10, dadurch gek..."
545094,EP17000010NWB1.xml,de\tSTREULICHTBLENDE EINES BILDERFASSUNGSGERÄT...,"Lens hood according to one of Claims 9 to 11, ...",Streulichtblende nach einem der Ansprüche 9 bi...
545095,EP17000010NWB1.xml,de\tSTREULICHTBLENDE EINES BILDERFASSUNGSGERÄT...,Lens hood according to one of the preceding cl...,Streulichtblende nach einem der vorherigen Ans...
545096,EP17000010NWB1.xml,de\tSTREULICHTBLENDE EINES BILDERFASSUNGSGERÄT...,Lens hood according to one of the preceding cl...,Streulichtblende nach einem der vorherigen Ans...


In [5]:
df.en[0]

'A modular power converter comprising a thermal support 12 for a circuit, the support comprising a substrate having fluid inlet 22 and outlet ports 24 and a circulation path 152 coupled between the inlet and outlet ports, the inlet and outlet ports being configured to transmit a coolant for circulation through the circulation path an interface plate 148 configured to support a power electronics circuit 130, the interface plate being configured for mounting to the support adjacent to the coolant circulation path for extraction of heat from the interface plate during operation anda power electronics circuit configured for power conversion supported on the interface plate anda driver circuit 34 for applying the drive signals to the power electronics circuit, the driver circuit being secured to and cooled by the thermal support wherein the support at least partially defines an electrical reference plane, a mechanical support, and a thermal extraction path for the power electronics circuit.

In [6]:
epo_corpus = corpus.Corpus(name='epo_en', 
                           doc=df,
                           tokenize_corpus=True,
                           load_vocab=True,
                          load_tokens=True)

17:40:47: Loading vocab
17:40:48: Loading tokens
17:40:58: Artefacts in the path: ../../data/word2vec/epo_en/
17:40:58: Sentences
	Lang	Size
	EN	545098
	DE	545098
17:40:58: Vocab
	Lang	Size
	EN	97354
	DE	392292


In [7]:
print(epo_corpus.all_toks_en[0], '\n', epo_corpus.all_toks_de[0])

['modular', 'power', 'converter', 'comprising', 'thermal', 'support', 'circuit', 'substrate', 'fluid', 'inlet', 'outlet', 'ports', 'circulation', 'path', 'coupled', 'configured', 'transmit', 'coolant', 'interface', 'plate', 'electronics', 'mounting', 'adjacent', 'extraction', 'heat', 'operation', 'anda', 'conversion', 'supported', 'driver', 'applying', 'drive', 'signals', 'secured', 'cooled', 'wherein', 'least', 'partially', 'defines', 'electrical', 'reference', 'plane', 'mechanical'] 
 ['modularer', 'leistungswandler', 'folgendes', 'umfasst', 'thermischen', 'träger', 'schaltung', 'wobei', 'substrat', 'fluideinlassanschlüssen', 'fluidauslassanschlüssen', 'zirkulationsweg', 'einlass', 'auslassanschlüsse', 'gekoppelt', 'konfiguriert', 'kühlmittel', 'zirkulation', 'durchzulassen', 'grenzflächenplatte', 'leistungselektronikschaltung', 'tragen', 'nähe', 'kühlmittelzirkulationswegs', 'montiert', 'betriebs', 'wärme', 'entnehmen', 'undeine', 'leistungsumwandlung', 'getragen', 'treiberschaltung

In [9]:
from gensim.models import FastText
from tqdm.notebook import tqdm as tq

In [10]:
def build_ft_vocab(ft):
    # Build vocab
    vocab = []
    for idx in tq(range(len(ft.wv))):
        vocab.append(ft.wv.index_to_key[idx])
    return vocab

In [ ]:
# Build EN model
ft_en = FastText(vector_size=300, 
                 window=5, 
                 min_count=1, 
                 sg=1)  # instantiate

data = epo_corpus.all_toks_en
ft_en.build_vocab(data)

ft_en.train(corpus_iterable=data, 
            total_examples=len(data), 
            epochs=5)

ft_en.save('../../data/word2vec/epo_en/models/fastText/fasttext_en.model')

In [11]:
en_vocab = build_ft_vocab(ft_en)

In [12]:
len(epo_corpus.all_toks_en), len(en_vocab)

(545098, 97354)

In [ ]:
# Build DE model
ft_de = FastText(vector_size=300, 
                 window=5, 
                 min_count=1, 
                 sg=1)  # instantiate

data = epo_corpus.all_toks_de
ft_de.build_vocab(data)

ft_de.train(corpus_iterable=data, 
            total_examples=len(data), 
            epochs=5)

ft_de.save('../../data/word2vec/epo_de/models/fastText/fasttext_de.model')
de_vocab = build_ft_vocab(ft_de)

In [11]:
# Load EN model

ft_en = FastText.load('../../data/word2vec/epo_en/models/fastText/fasttext_en.model')
en_vocab = build_ft_vocab(ft_en)


# Load DE model

ft_de = FastText.load('../../data/word2vec/epo_de/models/fastText/fasttext_de.model')
de_vocab = build_ft_vocab(ft_de)

17:41:31: loading FastText object from ../../data/word2vec/epo_en/models/fastText/fasttext_en.model
17:41:31: loading wv recursively from ../../data/word2vec/epo_en/models/fastText/fasttext_en.model.wv.* with mmap=None
17:41:31: loading vectors_vocab from ../../data/word2vec/epo_en/models/fastText/fasttext_en.model.wv.vectors_vocab.npy with mmap=None
17:41:31: loading vectors_ngrams from ../../data/word2vec/epo_en/models/fastText/fasttext_en.model.wv.vectors_ngrams.npy with mmap=None
17:41:31: setting ignored attribute vectors to None
17:41:31: setting ignored attribute buckets_word to None
17:41:39: FastText lifecycle event {'fname': '../../data/word2vec/epo_en/models/fastText/fasttext_en.model', 'datetime': '2021-09-05T17:41:39.269483', 'gensim': '4.0.1', 'python': '3.8.10 (default, Jun  2 2021, 10:49:15) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-81-generic-x86_64-with-glibc2.29', 'event': 'loaded'}


17:41:39: loading FastText object from ../../data/word2vec/epo_de/models/fastText/fasttext_de.model
17:41:39: loading wv recursively from ../../data/word2vec/epo_de/models/fastText/fasttext_de.model.wv.* with mmap=None
17:41:39: loading vectors_vocab from ../../data/word2vec/epo_de/models/fastText/fasttext_de.model.wv.vectors_vocab.npy with mmap=None


17:41:39: loading vectors_ngrams from ../../data/word2vec/epo_de/models/fastText/fasttext_de.model.wv.vectors_ngrams.npy with mmap=None
17:41:40: setting ignored attribute vectors to None
17:41:40: setting ignored attribute buckets_word to None
17:42:28: loading syn1neg from ../../data/word2vec/epo_de/models/fastText/fasttext_de.model.syn1neg.npy with mmap=None
17:42:28: setting ignored attribute cum_table to None
17:42:31: FastText lifecycle event {'fname': '../../data/word2vec/epo_de/models/fastText/fasttext_de.model', 'datetime': '2021-09-05T17:42:31.642726', 'gensim': '4.0.1', 'python': '3.8.10 (default, Jun  2 2021, 10:49:15) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-81-generic-x86_64-with-glibc2.29', 'event': 'loaded'}


## MUSE to align mono lingual embeddings

In [15]:
# Generate emb files to input to MUSE

from tqdm.notebook import tqdm as tq

def generate_emb_files(ft, vocab, lang):
    emb_file = open('ft_%s.emb'%(lang), 'w')
    emb_file.write('%d %d\n' %(len(vocab), 300))  
    for word in tq(vocab):
        # vec = w2v.model.wv[word]
        vec = ft.wv[word]
        vec_str = ''
        for i in vec:
            vec_str += str(i) + ' '
        emb_file.write(word+ ' ' + vec_str.strip()+ '\n')               

In [16]:
generate_emb_files(ft_en, en_vocab, 'en')

In [17]:
generate_emb_files(ft_de, de_vocab, 'de')

**Run MUSE and get the shared embeddings**

In [12]:
def get_vec_from_txt(path):
    file = open(path, 'r')

    word_vectors = {}
    ind = 0
    line = file.readline()
    while True:
        line = file.readline()
        if not line:
            break

        line_split = line.split(' ')
        word = line_split[0]
        vec = line_split[1:]

        vec_float = []
        for i in vec:
            vec_float.append(float(i))

        word_vectors[word] = vec_float

        print('ind: %s'%ind, end="\r")

        ind += 1
        
    file.close()
    print()
    
    return word_vectors

In [64]:
# Get wv from sup MUSE

en_wv_sup = get_vec_from_txt('../../data/word2vec/epo_en/projection/fastText/vectors-en.txt')
de_wv_sup = get_vec_from_txt('../../data/word2vec/epo_en/projection/fastText/vectors-de.txt')

ind: 97353
ind: 392291


In [65]:
len(en_wv_sup), len(de_wv_sup)

(97354, 392292)

In [13]:
# Get wv from unsup MUSE

en_wv_unsup = get_vec_from_txt('../../data/word2vec/epo_en/projection/fastText/vectors-en-unsup.txt')
de_wv_unsup = get_vec_from_txt('../../data/word2vec/epo_en/projection/fastText/vectors-de-unsup.txt')

ind: 97353
ind: 392291


In [14]:
len(en_wv_unsup), len(de_wv_unsup)

(97354, 392292)

## Projection

In [14]:
# Generate tsv files for Tensorflow projector

def generate_tsv_files(en_word_vectors, de_word_vectors):
    vec_file = open('vectors_epo_new.tsv', 'w')
    words_file = open('words_epo_new.tsv', 'w')
    
    for word in tq(en_word_vectors):
        # vec = w2v.model.wv[word]
        vec = en_word_vectors[word]
        vec_str = ''
        for i in vec:
            vec_str += str(i) + '\t'
        vec_file.write(vec_str.strip()+ '\n')
        words_file.write(word.strip()+ '\n')
        
    for word in tq(de_word_vectors):
        # vec = w2v.model.wv[word]
        vec = de_word_vectors[word]
        vec_str = ''
        for i in vec:
            vec_str += str(i) + '\t'
        vec_file.write(vec_str.strip()+ '\n')
        words_file.write(word.strip()+ '\n')
        
    vec_file.close()
    words_file.close()

In [23]:
generate_tsv_files(en_wv_sup, de_wv_sup)